In [258]:
import pandas as pd
from tqdm import tqdm

#### Разметка оригинального датасета на бинарную классификацию

In [266]:
original_df = pd.read_csv("data.csv")
original_df.head()

,market_sku_id,pic_url,verdict
0,1402035435,avatars.mds.yandex.net/get-marketpic/7044394/p...,clean_photo_good_background
1,1402035435,avatars.mds.yandex.net/get-marketpic/5620596/p...,clean_photo_good_background
2,1402035435,avatars.mds.yandex.net/get-marketpic/1708436/m...,other_infographics
3,729239006,avatars.mds.yandex.net/get-marketpic/1897710/p...,clean_photo_good_background
4,429050296,avatars.mds.yandex.net/get-marketpic/8786714/p...,good_infographics


In [267]:
new_label_name = {
    'clean_photo_good_background': 'clean_photo', 
    'other_infographics': "infographics",
    'good_infographics': "infographics", 
    'clean_photo_other_background': "clean_photo",
    'bad_infographics': "infographics", 
    'clean_photo_bad_background': 'clean_photo',
    'clean_photo_image_background': 'clean_photo'
}
tqdm.pandas()

original_df["verdict"] = original_df.verdict.progress_apply(lambda x: new_label_name[x]).drop(columns=["market_sku_id"])
original_df.verdict.value_counts()

100%|██████████| 1131832/1131832 [00:00<00:00, 2527775.05it/s]


infographics    571194
clean_photo     560638
Name: verdict, dtype: int64

In [268]:
original_df.head()

,market_sku_id,pic_url,verdict
0,1402035435,avatars.mds.yandex.net/get-marketpic/7044394/p...,clean_photo
1,1402035435,avatars.mds.yandex.net/get-marketpic/5620596/p...,clean_photo
2,1402035435,avatars.mds.yandex.net/get-marketpic/1708436/m...,infographics
3,729239006,avatars.mds.yandex.net/get-marketpic/1897710/p...,clean_photo
4,429050296,avatars.mds.yandex.net/get-marketpic/8786714/p...,infographics


#### Оставили только рабочие ссылки и получили csv

(полный код в create_new_csv.ipynb, это делали делали в облаке, результат сохранен в data_correct.csv)

In [98]:
correct_links = pd.read_csv("data_correct.csv")
print(correct_links.shape)
correct_links.head()

(1104969, 2)


,path,verdict
0,/home/jupyter/mnt/datasets/pictures/pic_0.jpg,clean_photo_good_background
1,/home/jupyter/mnt/datasets/pictures/pic_1.jpg,clean_photo_good_background
2,/home/jupyter/mnt/datasets/pictures/pic_2.jpg,other_infographics
3,/home/jupyter/mnt/datasets/pictures/pic_3.jpg,clean_photo_good_background
4,/home/jupyter/mnt/datasets/pictures/pic_4.jpg,good_infographics


#### Формирование выборки для бинарной классификации: 

есть ли инфографика или нет инфографики (чистое фото)

In [99]:
label_names = correct_links.verdict.unique()
label_names

array(['clean_photo_good_background', 'other_infographics',
       'good_infographics', 'clean_photo_other_background',
       'bad_infographics', 'clean_photo_bad_background',
       'clean_photo_image_background'], dtype=object)

In [159]:
new_label_name = {
    'clean_photo_good_background': 'clean_photo', 
    'other_infographics': "infographics",
    'good_infographics': "infographics", 
    'clean_photo_other_background': "clean_photo",
    'bad_infographics': "infographics", 
    'clean_photo_bad_background': 'clean_photo',
    'clean_photo_image_background': 'clean_photo'
}
tqdm.pandas()
frame_for_binary = correct_links.copy()
frame_for_binary


name_label_bin = ['other_infographics', 'good_infographics', 'bad_infographics']
csv_only_bin = pd.DataFrame({"path": [], "verdict": []})
for name in name_label_bin:
    csv_only_bin = pd.concat([ 
        csv_only_bin,
        frame_for_binary[frame_for_binary.verdict == name].sample(2000*5, random_state=17)
              ])

name_label_bin = ['clean_photo_good_background',
   'clean_photo_other_background',
   'clean_photo_bad_background',
   'clean_photo_image_background'
    ]   
for name in name_label_bin:
    csv_only_bin = pd.concat([ 
        csv_only_bin,
        frame_for_binary[frame_for_binary.verdict == name].sample(1_500*5, random_state=17)
              ])

csv_only_bin["verdict"] = csv_only_bin.verdict.progress_apply(lambda x: new_label_name[x])
print(csv_only_bin.shape[0])
csv_only_bin.to_csv("data_for_binary.csv", index=False)
csv_only_bin.head()

100%|██████████| 60000/60000 [00:00<00:00, 2226433.58it/s]

60000


,path,verdict
1070352,/home/jupyter/mnt/datasets/pictures/pic_109643...,infographics
117919,/home/jupyter/mnt/datasets/pictures/pic_121318...,infographics
481146,/home/jupyter/mnt/datasets/pictures/pic_495092...,infographics
1014823,/home/jupyter/mnt/datasets/pictures/pic_103978...,infographics
294229,/home/jupyter/mnt/datasets/pictures/pic_302544...,infographics


In [161]:
csv_only_bin.verdict.value_counts()

infographics    30000
clean_photo     30000
Name: verdict, dtype: int64

#### Формирование выборки только для инфографики

In [103]:
only_infgr = correct_links.copy()
only_infgr = only_infgr[only_infgr.verdict.isin([
   'other_infographics',
   'good_infographics',
   'bad_infographics'
])]
only_infgr.verdict.value_counts()

good_infographics     292314
other_infographics    215554
bad_infographics       49625
Name: verdict, dtype: int64

In [104]:
name_label_gr = only_infgr.verdict.unique()
csv_only_infgr = pd.DataFrame({"path": [], "verdict": []})
for name in name_label_gr:
    csv_only_infgr = pd.concat([ 
        csv_only_infgr,
        only_infgr[only_infgr.verdict == name].sample(10_000, random_state=42)
              ])

In [192]:
60_000 / 20

3000.0

In [193]:
200_000 / 3000.0

66.66666666666667

In [105]:
print(csv_only_infgr.verdict.value_counts())
csv_only_infgr.to_csv("data_for_infographics.csv", index=False)

other_infographics    10000
good_infographics     10000
bad_infographics      10000
Name: verdict, dtype: int64


#### Формирование выборки только для фона (чистое фото)

In [106]:
only_photo = correct_links.copy()
only_photo = only_photo[only_photo.verdict.isin([
   'clean_photo_good_background',
   'clean_photo_other_background',
   'clean_photo_bad_background',
   'clean_photo_image_background'
])]
only_photo.verdict.value_counts()

clean_photo_good_background     455307
clean_photo_other_background     49569
clean_photo_bad_background       28093
clean_photo_image_background     14507
Name: verdict, dtype: int64

In [107]:
name_label_ph = only_photo.verdict.unique()
csv_only_ph = pd.DataFrame({"path": [], "verdict": []})
for name in name_label_ph:
    csv_only_ph = pd.concat([ 
        csv_only_ph,
        only_photo[only_photo.verdict == name].sample(10_000, random_state=42)
              ])

In [108]:
print(csv_only_infgr.verdict.value_counts())
csv_only_ph.to_csv("data_for_background.csv", index=False)

other_infographics    10000
good_infographics     10000
bad_infographics      10000
Name: verdict, dtype: int64


In [109]:
pd.read_csv("data_for_background.csv").verdict.unique()

array(['clean_photo_good_background', 'clean_photo_other_background',
       'clean_photo_bad_background', 'clean_photo_image_background'],
      dtype=object)

In [ ]:
pd.read_csv("data_for_infographics.csv").verdict.unique()

array(['other_infographics', 'good_infographics', 'bad_infographics'],
      dtype=object)